# Cluster the neighborhoods in the city of Toronto


Installing neccessary packages

In [1]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


#### Step 01. Importing the packages

In [4]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

#### Step 02. Requesting data from the Wikipidea site

In [5]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [17]:
source = requests.get(url).text
soup = BeautifulSoup(source, 'lxml')

#### Step 03. Extract the wikitable with neighborhood data

In [7]:
wiki_table = soup.find('table')
print(wiki_table.prettify())

<table class="wikitable">
 <tbody>
  <tr>
   <th>
    Postal code
   </th>
   <th>
    Borough
   </th>
   <th>
    Neighborhood
   </th>
  </tr>
  <tr>
   <td>
    M1A
   </td>
   <td>
    Not assigned
   </td>
   <td>
   </td>
  </tr>
  <tr>
   <td>
    M2A
   </td>
   <td>
    Not assigned
   </td>
   <td>
   </td>
  </tr>
  <tr>
   <td>
    M3A
   </td>
   <td>
    North York
   </td>
   <td>
    Parkwoods
   </td>
  </tr>
  <tr>
   <td>
    M4A
   </td>
   <td>
    North York
   </td>
   <td>
    Victoria Village
   </td>
  </tr>
  <tr>
   <td>
    M5A
   </td>
   <td>
    Downtown Toronto
   </td>
   <td>
    Regent Park / Harbourfront
   </td>
  </tr>
  <tr>
   <td>
    M6A
   </td>
   <td>
    North York
   </td>
   <td>
    Lawrence Manor / Lawrence Heights
   </td>
  </tr>
  <tr>
   <td>
    M7A
   </td>
   <td>
    Downtown Toronto
   </td>
   <td>
    Queen's Park / Ontario Provincial Government
   </td>
  </tr>
  <tr>
   <td>
    M8A
   </td>
   <td>
    Not assigned
   </

In [27]:
table_rows = wiki_table.find_all('tr')
wiki_output=[]


for tr in table_rows:
    td=tr.find_all('td')
    wiki_data=[]
    for column in td:
        wiki_data.append(column.text)
    wiki_output.append(wiki_data)

wiki_pd=pd.DataFrame(wiki_output, columns =['PostalCode','Borough','Neighborhood'])    

wiki_pd=wiki_pd.drop(wiki_pd.index[0])

wiki_pd = wiki_pd.replace('\n','', regex=True)

wiki_pd.head()



,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,
2,M2A,Not assigned,
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Regent Park / Harbourfront


In [28]:
wiki_pd.shape

(180, 3)